<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/MEPS_(MetCoOp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install xarray pydap netCDF4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 140.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.9 MB/s eta 0:00:00


In [12]:
import pandas as pd
import requests
import io
import time

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

print("Fetching 2024 MEPS Archive: GHI, Wind & Temp...")

for city, coords in cities.items():
    # Variable 'shortwave_radiation' in MEPS = GHI
    url = (
        f"https://historical-forecast-api.open-meteo.com/v1/forecast?"
        f"latitude={coords['lat']}&longitude={coords['lon']}&"
        f"start_date=2024-01-01&end_date=2024-12-31&"
        f"hourly=shortwave_radiation,temperature_2m,wind_speed_10m&"
        f"models=metno_nordic&"
        f"wind_speed_unit=ms&format=csv"
    )

    try:
        response = requests.get(url)

        if response.status_code != 200:
            print(f"❌ Error {response.status_code} for {city}")
            continue

        df = pd.read_csv(io.StringIO(response.text), skiprows=3)

        if df.empty:
            print(f"⚠️ No data for {city}")
            continue

        # Rename to clear solar energy terms
        df = df.rename(columns={
            'shortwave_radiation (W/m²)': 'GHI (W/m2)',
            'temperature_2m (°C)': 'Air Temp (C)',
            'wind_speed_10m (m/s)': 'Wind Speed (m/s)'
        })

        filename = f"{city.lower()}_meps_full_2024.csv"
        df.to_csv(filename, index=False)
        print(f"✅ {city} saved with GHI, Temp, and Wind!")

        time.sleep(1)

    except Exception as e:
        print(f"❌ Failed {city}: {e}")

print("\nData collection complete.")

Fetching 2024 MEPS Archive: GHI, Wind & Temp...
✅ Turku saved with GHI, Temp, and Wind!
✅ Copenhagen saved with GHI, Temp, and Wind!
✅ Stockholm saved with GHI, Temp, and Wind!
✅ Oslo saved with GHI, Temp, and Wind!

Data collection complete.
